# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import joblib
from azureml.core import Model
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import InferenceConfig
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.core import Dataset, Environment, Experiment, Workspace
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.core.compute import ComputeTarget, AmlCompute

## Dataset

Data were extracted from images that were taken from genuine and forged banknote-like specimens. For digitization, an industrial camera usually used for print inspection was used. The final images have 400x 400 pixels. Due to the object lens and distance to the investigated object gray-scale pictures with a resolution of about 660 dpi were gained. Wavelet Transform tool were used to extract features from images.

It is a classification problem to find out whether the bank note is genuine or forged.

The above note and dataset have been collected from [UCI](https://archive.ics.uci.edu/ml/datasets/banknote+authentication).

In [ ]:
ws = Workspace.from_config()

dataset_name = "data_banknote_authentication.csv"

#Getting dataset that is registered onto workspace
dataset = Dataset.get_by_name(ws, dataset_name)

train_data = dataset.to_pandas_dataframe()
print("Shape : ", str(train_data.shape))
train_data.head()

label = 'Class'

# Name of the experiment
experiment_name = 'azure_ml_capstone_project'
experiment=Experiment(ws, experiment_name)

experiment

## Create compute instance

In [ ]:
# Choose a name for your CPU cluster
amlcompute_name = "aml-compute"

# Verify that cluster does not exist already
try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',
                                                           max_nodes=7)
    aml_compute = ComputeTarget.create(ws, amlcompute_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

This is a classification problem, and I have made use of Logistic Regression for this task. Two parameters are tuned, and they are as follows.

* C: is the inverse of the regularization term (1/lambda). It tells the model how much large parameters are penalized, smaller values result in larger penalization. We can think of regularization as adding (or increasing the) bias if our model suffers from (high) variance (i.e., it overfits the training data). On the other hand, too much bias will result in underfitting (a characteristic indicator of high bias is that the model shows a "bad" performance for both the training and test dataset).

* max_iter: is the maximum iteration to converge.

Termination policy: 

I have used here bandit policy. It terminates runs where the primary metric is not within the specified slack factor/slack amount compared to the best performing run. It saves computation time.

Config settings: 

 * estimator: An estimator is called with sampled hyperparameters.
 * policy: Bandit policy is configured here. 
 * primary_metric_name : Name of metric for which runs will be evaluated.
 * primary_metric_goal: It determines whether primary metric will be maximized or minimized.
 * max_total_runs: The maximum total number of runs to create.
 * max_concurrent_runs: The maximum number of runs to execute concurrently.
 
 [Parmeter explanation](https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive.hyperdriveconfig?view=azure-ml-py)

In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=1, slack_factor=0.3, delay_evaluation=4)

#TODO: Create the different params that you will be using during training
ps = RandomParameterSampling(
                                {
                                    "--C": inform(0.1,1), "--max_iter": choice(75,100,125,150)
                                }
                            )


# Code below makes a new directory for training
if "training" not in os.listdir():
    os.mkdir("./training")

#copying train.py to training directory    
import shutil
shutil.copy('train.py', './training')

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory='./training', compute_target=aml_compute, entry_script='train.py', script_params={'--input_data': dataset_name})

hyperdrive_run_config = HyperDriveConfig(
    estimator=estimator,
    hyperparameter_sampling=param_sampling,
    policy=early_termination_policy,
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=15,
    max_concurrent_runs=6
)

In [ ]:
#TODO: Submit your experiment
bank_note_hyperdrive_run = experiment.submit(hyperdrive_run_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(bank_note_hyperdrive_run).show()
bank_note_hyperdrive_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run = bank_note_hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.id)
best_run.get_metrics()

In [ ]:
#TODO: Save the best model
best_run.download_file(name="outputs/model.joblib", output_file_path="./hyperdrive_model.joblib")

# Register
model = best_run.register_model(
    model_name='hd_model', 
    model_path='outputs/model.joblib',
)


In [ ]:
joblib.load("./hyperdrive_model.joblib")

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
env = Environment.get(ws, "AzureML-Minimal").clone("udacity_ml")

for pip_pkg in ["scikit-learn"]:
    env.python.conda_dependencies.add_pip_package(pip_pkg)

inference_config = InferenceConfig(entry_script='score.py',
                                    environment=env)

env.save_to_directory("./env/", overwrite=True)

In [ ]:
deployment_config = AciWebservice.deploy_configuration(cpu_cores=2, memory_gb=3,
                                                enable_app_insights=True, auth_enabled=True) 

service_name = 'bank-note-authentication-service'
service = Model.deploy(ws, service_name, [model], inference_config=inference_config, 
                       deployment_config=deployment_config, overwrite=True)
service.wait_for_deployment(show_output = True)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
x_test = train_data.iloc[120, :-1]
y_test = iris.loc[120, "Class"]
print("X test: " + x_test.toList())
print("Y test: " + y_test)

In [ ]:
import json
payload = json.dumps(x_test)

output = service.run(payload)
print(f"Predicted: {output}\Original: {y_test}")

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.get_logs())

# Clean up

In [ ]:
service.delete()
aml_compute.delete()